In [2]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

# load data

In [162]:
df_ct = pd.read_excel('/root/yubin/CoCONet/data/CT.xlsx')
df_mri = pd.read_excel('/root/yubin/CoCONet/data/mri_key.xlsx')

In [4]:
# baseline, follow income
df_base = pd.read_excel('/root/yubin/CoCONet/data/baseline_with_allrace.xlsx')
df_follow = pd.read_excel('/root/yubin/CoCONet/data/followup.xlsx')

## Cortical Thickness

### pair check

In [262]:
pair = df_ct['subjectkey'].value_counts() == 2

In [263]:
# pair 이루는 subjectkey
mask = df_ct['subjectkey'].isin(pair.index[:7776])

In [264]:
df_ct = df_ct[mask].reset_index(drop=True)

In [265]:
print('shape:',df_ct.shape)
print('# of subject:',len(df_ct['subjectkey'].unique()))

shape: (15552, 155)
# of subject: 7776


### diff

In [266]:
# 이름 - eventname 정렬 (base-follow -> ascending=True, follow-base False)
df_ct.sort_values(['subjectkey', 'eventname'], ascending=False, inplace=True)
df_ct.reset_index(inplace=True, drop=True)

In [267]:
df_ct

,collection_id,abcd_mrisdp10201_id,subjectkey,src_subject_id,interview_age,sex,eventname,mrisdp_1,mrisdp_2,mrisdp_3,...,mrisdp_139,mrisdp_140,mrisdp_141,mrisdp_142,mrisdp_143,mrisdp_144,mrisdp_145,mrisdp_146,mrisdp_147,mrisdp_148
0,2573,18763,NDAR_INVZZZP87KR,NDAR_INVZZZP87KR,126,F,baseline_year_1_arm_1,2.666,2.288,2.768,...,2.341,1.595,2.321,2.751,2.654,2.56,2.618,2.526,2.779,3.026
1,2573,18736,NDAR_INVZZZP87KR,NDAR_INVZZZP87KR,150,F,2_year_follow_up_y_arm_1,2.48,2.149,2.494,...,2.108,1.673,2.239,2.793,2.607,1.986,2.784,2.483,2.664,2.735
2,2573,18710,NDAR_INVZZZ2ALR6,NDAR_INVZZZ2ALR6,121,F,baseline_year_1_arm_1,2.464,2.753,2.658,...,2.55,1.734,2.415,2.632,2.502,2.581,2.66,2.4,2.76,2.477
3,2573,18709,NDAR_INVZZZ2ALR6,NDAR_INVZZZ2ALR6,145,F,2_year_follow_up_y_arm_1,2.352,2.715,2.577,...,2.505,1.656,2.386,2.616,2.577,2.219,2.579,2.463,2.719,2.527
4,2573,18684,NDAR_INVZZPKBDAC,NDAR_INVZZPKBDAC,113,F,baseline_year_1_arm_1,2.608,2.825,2.714,...,2.562,1.938,2.707,2.875,2.56,2.997,2.679,2.782,2.756,3.083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15547,2573,3,NDAR_INV00HEV6HB,NDAR_INV00HEV6HB,149,M,2_year_follow_up_y_arm_1,2.36,2.433,2.372,...,2.234,1.655,2.092,2.48,2.472,3.039,2.489,2.61,2.652,2.277
15548,2573,2,NDAR_INV00CY2MDM,NDAR_INV00CY2MDM,130,M,baseline_year_1_arm_1,2.551,2.509,2.842,...,2.537,1.849,2.477,2.692,2.633,2.492,2.641,2.499,2.74,2.5
15549,2573,1,NDAR_INV00CY2MDM,NDAR_INV00CY2MDM,152,M,2_year_follow_up_y_arm_1,2.564,2.472,2.86,...,2.482,1.876,2.445,2.637,2.616,2.406,2.58,2.514,2.727,2.561
15550,2573,7124,NDARMC003PZF,NDAR_INVBKW33JVR,123,F,baseline_year_1_arm_1,2.369,2.259,2.31,...,2.43,1.588,2.25,2.575,2.519,2.723,2.433,2.354,2.744,2.604


In [268]:
# type 변경
ct_cols = ['mrisdp_' + str(i) for i in range(1, 149)]
roi_dict = pd.DataFrame()
roi_dict['col_name'] = ct_cols
roi_dict['type'] = float

# roi CT 값들 float
type_dict = roi_dict.set_index('col_name').T.to_dict('records')[0]
# interview age
type_dict['interview_age'] = int


df_ct = df_ct.astype(type_dict)

In [269]:
# follow - baseline
df_diff = df_ct[ct_cols].diff()
df_diff = df_diff.iloc[1::2, :].reset_index(drop=True)

In [270]:
df_diff

,mrisdp_1,mrisdp_2,mrisdp_3,mrisdp_4,mrisdp_5,mrisdp_6,mrisdp_7,mrisdp_8,mrisdp_9,mrisdp_10,...,mrisdp_139,mrisdp_140,mrisdp_141,mrisdp_142,mrisdp_143,mrisdp_144,mrisdp_145,mrisdp_146,mrisdp_147,mrisdp_148
0,-0.186,-0.139,-0.274,-0.115,-0.092,-0.048,-0.165,-0.075,-0.156,-0.066,...,-0.233,0.078,-0.082,0.042,-0.047,-0.574,0.166,-0.043,-0.115,-0.291
1,-0.112,-0.038,-0.081,-0.121,-0.091,-0.039,-0.102,-0.189,-0.120,-0.147,...,-0.045,-0.078,-0.029,-0.016,0.075,-0.362,-0.081,0.063,-0.041,0.050
2,-0.042,-0.069,-0.058,0.026,-0.050,0.041,0.121,-0.080,-0.077,0.051,...,-0.043,0.028,-0.090,-0.050,-0.057,0.130,-0.061,0.028,0.045,0.039
3,0.059,0.003,0.185,-0.030,0.052,-0.075,0.045,-0.087,-0.082,-0.203,...,-0.008,-0.063,0.070,0.065,0.015,-0.147,0.016,-0.062,0.007,-0.022
4,-0.034,-0.094,-0.072,-0.086,-0.052,-0.040,0.017,-0.037,-0.006,-0.240,...,-0.025,0.042,-0.042,-0.015,-0.030,-0.026,-0.043,0.001,-0.064,-0.059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7771,-0.112,0.050,0.051,-0.083,-0.045,-0.038,-0.062,-0.148,0.045,-0.137,...,0.020,0.021,0.059,0.026,0.026,0.212,0.025,-0.024,-0.014,0.001
7772,0.015,-0.083,-0.024,0.008,0.150,0.063,0.048,-0.003,-0.097,0.025,...,-0.069,0.045,-0.030,-0.036,-0.023,0.251,0.016,0.005,-0.006,-0.109
7773,0.089,-0.077,-0.081,-0.062,0.032,-0.062,-0.058,-0.040,0.083,0.085,...,-0.085,0.082,-0.105,-0.059,-0.120,-0.004,-0.115,-0.104,-0.067,-0.221
7774,0.013,-0.037,0.018,0.032,-0.078,0.031,-0.033,-0.030,-0.171,0.018,...,-0.055,0.027,-0.032,-0.055,-0.017,-0.086,-0.061,0.015,-0.013,0.061


In [271]:
def CT_diff(original_df, diff_df):
    rows, cols = original_df.shape
    subjects = int(rows/2)

    return_df = pd.DataFrame(columns=['subjectkey', 'age_x', 'age_y', 'gender'])

    for s_i in range(subjects):
        subject_info = [
            original_df['subjectkey'][2*s_i], # subjectkey
            original_df['interview_age'][2*s_i],  # age_x(base_age)
            original_df['interview_age'][2*s_i+1], # age_y(2year_age)
            0 if original_df['sex'][2*s_i]=='F' else 1
            ]
        return_df.loc[return_df.shape[0]] = subject_info

    return pd.concat([return_df, diff_df], axis=1)

In [272]:
df_ct_diff = CT_diff(df_ct, df_diff)
df_ct_diff

,subjectkey,age_x,age_y,gender,mrisdp_1,mrisdp_2,mrisdp_3,mrisdp_4,mrisdp_5,mrisdp_6,...,mrisdp_139,mrisdp_140,mrisdp_141,mrisdp_142,mrisdp_143,mrisdp_144,mrisdp_145,mrisdp_146,mrisdp_147,mrisdp_148
0,NDAR_INVZZZP87KR,126,150,0,-0.186,-0.139,-0.274,-0.115,-0.092,-0.048,...,-0.233,0.078,-0.082,0.042,-0.047,-0.574,0.166,-0.043,-0.115,-0.291
1,NDAR_INVZZZ2ALR6,121,145,0,-0.112,-0.038,-0.081,-0.121,-0.091,-0.039,...,-0.045,-0.078,-0.029,-0.016,0.075,-0.362,-0.081,0.063,-0.041,0.050
2,NDAR_INVZZPKBDAC,113,136,0,-0.042,-0.069,-0.058,0.026,-0.050,0.041,...,-0.043,0.028,-0.090,-0.050,-0.057,0.130,-0.061,0.028,0.045,0.039
3,NDAR_INVZZNX6W2P,131,156,1,0.059,0.003,0.185,-0.030,0.052,-0.075,...,-0.008,-0.063,0.070,0.065,0.015,-0.147,0.016,-0.062,0.007,-0.022
4,NDAR_INVZZLZCKAY,110,131,0,-0.034,-0.094,-0.072,-0.086,-0.052,-0.040,...,-0.025,0.042,-0.042,-0.015,-0.030,-0.026,-0.043,0.001,-0.064,-0.059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7771,NDAR_INV00LJVZK2,121,147,1,-0.112,0.050,0.051,-0.083,-0.045,-0.038,...,0.020,0.021,0.059,0.026,0.026,0.212,0.025,-0.024,-0.014,0.001
7772,NDAR_INV00LH735Y,109,133,1,0.015,-0.083,-0.024,0.008,0.150,0.063,...,-0.069,0.045,-0.030,-0.036,-0.023,0.251,0.016,0.005,-0.006,-0.109
7773,NDAR_INV00HEV6HB,124,149,1,0.089,-0.077,-0.081,-0.062,0.032,-0.062,...,-0.085,0.082,-0.105,-0.059,-0.120,-0.004,-0.115,-0.104,-0.067,-0.221
7774,NDAR_INV00CY2MDM,130,152,1,0.013,-0.037,0.018,0.032,-0.078,0.031,...,-0.055,0.027,-0.032,-0.055,-0.017,-0.086,-0.061,0.015,-0.013,0.061


## MRI
0, 1로 원핫인코딩

In [328]:
df_mri = pd.read_excel('/root/yubin/CoCONet/data/mri_key.xlsx')

In [329]:
# data 설명 제거
df_mri.drop(0, inplace=True)

In [330]:
df_mri['mri_info_manufacturer'].value_counts()

SIEMENS                    7278
GE MEDICAL SYSTEMS         2962
Philips Medical Systems    1512
Name: mri_info_manufacturer, dtype: int64

In [331]:
siemens_arr = df_mri['mri_info_manufacturer']=='SIEMENS'
ge_arr = df_mri['mri_info_manufacturer']=='GE MEDICAL SYSTEMS'
philips_arr = df_mri['mri_info_manufacturer']=='Philips Medical Systems'

In [332]:
df_mri['GE'] = ge_arr*1
df_mri['Philips'] = philips_arr*1
df_mri['SIEMENS'] = siemens_arr*1

df_mri.drop('mri_info_manufacturer', axis=1, inplace=True)

In [333]:
df_mri.reset_index(inplace=True, drop=True)
df_mri

,key,GE,Philips,SIEMENS
0,NDAR_INVUK2F9WW4,1,0,0
1,NDAR_INVE46BJ2YA,0,0,1
2,NDAR_INVHWX750UV,0,0,1
3,NDAR_INVHXHUVGJG,0,1,0
4,NDAR_INVDPYR4KM2,0,0,1
...,...,...,...,...
11747,NDAR_INVW0BM9LJ7,1,0,0
11748,NDAR_INVW26YNDL9,0,0,1
11749,NDAR_INVW4PDFHV1,0,1,0
11750,NDAR_INVFJANMBLW,0,0,1


## base follow merge

In [39]:
df_follow2 = df_follow[df_follow['eventname']=='2_year_follow_up_y_arm_1']

In [230]:
df_income = pd.merge(df_base, df_follow2, on=['subjectkey', 'gender'])
df_income.drop(['eventname_x', 'eventname_y', 'interview_age_x', 'interview_age_y', 'gender'], axis=1, inplace=True)

In [247]:
# up: 1 down: 0 same: -1
df_income.loc[df_income['income_x'] > df_income['income_y'], 'income'] = 0 # down
df_income.loc[df_income['income_x'] < df_income['income_y'], 'income'] = 1 # up
df_income.loc[df_income['income_x'] == df_income['income_y'], 'income'] = -1 # same

In [250]:
df_income = df_income.astype({'income_x':int, 'income':int, 'White':int, 'Black':int, 'Others':int})
df_income

,subjectkey,income_x,White,Black,Others,income_y,income
0,NDAR_INV0A4ZDYNL,8,1,1,1,8,-1
1,NDAR_INV0BKE31EY,10,1,0,0,10,-1
2,NDAR_INV0CBFTKR7,9,0,1,0,9,-1
3,NDAR_INV0DC9BJZK,7,1,0,0,7,-1
4,NDAR_INV0EV57VEX,9,1,0,0,9,-1
...,...,...,...,...,...,...,...
8965,NDAR_INVZXF5C635,10,1,0,0,10,-1
8966,NDAR_INVZYLV9BMB,7,0,0,1,7,-1
8967,NDAR_INVZZPKBDAC,10,1,0,0,9,0
8968,NDAR_INV59BE4FA2,8,1,0,0,10,1


# Merge

In [334]:
df = pd.merge(df_income, df_mri, left_on='subjectkey', right_on='key')
df = pd.merge(df, df_ct_diff, on='subjectkey')

# null있는 행 제거
df.dropna()
df.drop('key', axis=1, inplace=True)

df

,subjectkey,income_x,White,Black,Others,income_y,income,GE,Philips,SIEMENS,...,mrisdp_139,mrisdp_140,mrisdp_141,mrisdp_142,mrisdp_143,mrisdp_144,mrisdp_145,mrisdp_146,mrisdp_147,mrisdp_148
0,NDAR_INV0A4ZDYNL,8,1,1,1,8,-1,0,0,1,...,-0.138,0.051,-0.015,-0.070,-0.024,0.058,-0.043,-0.055,-0.131,-0.163
1,NDAR_INV0CBFTKR7,9,0,1,0,9,-1,0,1,0,...,-0.126,-0.047,-0.125,-0.190,-0.133,-1.013,0.091,-0.012,-0.132,0.045
2,NDAR_INV0DC9BJZK,7,1,0,0,7,-1,0,0,1,...,-0.026,-0.121,-0.031,-0.098,-0.024,-0.007,-0.053,-0.136,-0.046,0.008
3,NDAR_INV0EV57VEX,9,1,0,0,9,-1,0,0,1,...,-0.117,-0.053,-0.054,-0.001,-0.060,-0.151,-0.094,0.003,-0.051,0.107
4,NDAR_INV0GPKYMDC,6,1,0,0,4,0,0,0,1,...,-0.026,0.003,0.070,-0.008,0.076,-0.039,-0.065,-0.094,-0.046,-0.168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6762,NDAR_INVZVD13ZMG,9,1,0,1,9,-1,0,1,0,...,-0.016,-0.064,0.038,-0.023,0.025,-0.032,-0.037,-0.132,-0.009,0.201
6763,NDAR_INVZWE6CD6Y,7,1,0,0,7,-1,1,0,0,...,0.014,0.045,-0.009,0.208,-0.093,0.116,-0.045,0.063,-0.041,-0.077
6764,NDAR_INVZXF5C635,10,1,0,0,10,-1,0,0,1,...,-0.021,-0.010,0.021,-0.003,-0.012,-0.139,-0.031,-0.019,-0.047,0.016
6765,NDAR_INVZYLV9BMB,7,0,0,1,7,-1,0,1,0,...,0.010,0.030,0.001,-0.091,-0.022,-0.092,-0.018,-0.096,-0.032,-0.149


In [300]:
df['income'].value_counts() # 1802, 686

-1    3877
 1    2074
 0     816
Name: income, dtype: int64

In [314]:
df['gender'].value_counts() # N=5913 subjects (male:female=3172:2741)

1    3645
0    3122
Name: gender, dtype: int64

# FA vs. CT

In [301]:
df_fa = pd.read_excel('/root/yubin/CoCONet/data/base_UD.xlsx')

In [311]:
fa_list = set(df_fa['subjectkey'])
ct_list = set(df[df['income']!=-1]['subjectkey'])

In [312]:
print('intersection:',len(fa_list&ct_list))
print('FA-CT:', len(fa_list-ct_list))
print('CT-FA:', len(ct_list-fa_list))

intersection: 2488
FA-CT: 0
CT-FA: 402


## FA 기준

In [335]:
df_2488 = df[df['subjectkey'].isin(fa_list)].reset_index(drop=True)
df_2488

,subjectkey,income_x,White,Black,Others,income_y,income,GE,Philips,SIEMENS,...,mrisdp_139,mrisdp_140,mrisdp_141,mrisdp_142,mrisdp_143,mrisdp_144,mrisdp_145,mrisdp_146,mrisdp_147,mrisdp_148
0,NDAR_INV0GPKYMDC,6,1,0,0,4,0,0,0,1,...,-0.026,0.003,0.070,-0.008,0.076,-0.039,-0.065,-0.094,-0.046,-0.168
1,NDAR_INV0HL5D3DF,5,0,1,0,7,1,0,0,1,...,-0.114,0.001,-0.540,-0.782,-0.928,-0.352,-0.176,-0.029,-0.150,-0.352
2,NDAR_INV0P3ERKVG,9,1,0,0,10,1,1,0,0,...,-0.054,0.013,-0.070,0.009,-0.036,0.019,0.119,0.046,-0.041,0.196
3,NDAR_INV0U23M45G,8,1,0,0,9,1,0,0,1,...,0.027,0.022,-0.004,0.005,0.039,-0.109,-0.035,0.014,-0.011,0.056
4,NDAR_INV0UV5WZUN,8,1,0,0,7,0,0,0,1,...,-0.029,0.001,-0.123,-0.047,0.078,-0.300,0.017,0.055,-0.100,0.051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2483,NDAR_INVZG6365BX,9,1,0,0,10,1,0,0,1,...,-0.061,-0.117,-0.034,-0.077,-0.071,-0.117,0.043,-0.038,-0.022,-0.053
2484,NDAR_INVZGEEAXPZ,3,1,0,0,5,1,0,0,1,...,-0.036,-0.026,-0.040,0.433,0.832,-0.049,0.014,-0.071,-0.035,-0.118
2485,NDAR_INVZNXT0TX2,9,0,0,1,6,0,1,0,0,...,-0.065,-0.003,0.101,0.037,0.042,-0.315,0.174,-0.011,-0.033,0.001
2486,NDAR_INVZTWTFU2A,3,0,1,0,4,1,1,0,0,...,-0.059,-0.098,-0.136,-0.080,-0.011,-0.298,-0.220,-0.074,-0.074,0.148


In [321]:
df_2488.groupby('income').sum()

/tmp/ipykernel_98701/401868361.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_2488.groupby('income').sum()


,income_x,White,Black,Others,income_y,GE,Philips,SIMENS,age_x,age_y,...,mrisdp_139,mrisdp_140,mrisdp_141,mrisdp_142,mrisdp_143,mrisdp_144,mrisdp_145,mrisdp_146,mrisdp_147,mrisdp_148
income,,,,,,,,,,,,,,,,,,,,,
0,4915,499,187,104,3853,200,88,398,81568,98410,...,-35.904,-14.899,-29.677,-11.963,-3.079,-48.164,-32.577,-19.671,-28.643,-22.651
1,11305,1365,370,290,13795,448,193,1161,214020,258181,...,-88.589,-31.193,-78.145,-22.776,-6.444,-120.044,-84.794,-47.779,-72.849,-42.500


In [319]:
df_2488['income'].value_counts()

1    1802
0     686
Name: income, dtype: int64

In [336]:
df_2488.to_excel('/root/yubin/CoCONet/data/CT_UD.xlsx')

# GLM

In [325]:
import pandas as pd
import os
from scipy.stats import f
from scipy import stats
import numpy as np
import torch
import matplotlib.pyplot as plt

In [357]:
def get_XYZ(df):
    cols = ['mrisdp_' + str(i) for i in range(1, 149)] #['ddtidp_' + str(i) for i in range(605, 605+148)]
    Y = df[cols] 
    Z = df[['age_x', 'age_y', 'gender', 'GE', 'Philips', 'SIEMENS', 'White', 'Black', 'Others']] 
    X = df[['income']] 

    return X, Y, Z

def GLM(y, Z, X):
    n = y.shape[0]
    k = Z.shape[1] + 1
    p = X.shape[1] 

    # reduced model: y = Z * lambda
    W = torch.cat((torch.ones(n).reshape(-1, 1), Z), dim=1)
    lda = torch.matmul(torch.matmul(torch.pinverse(torch.matmul(W.T, W)), W.T), y)
    SSE0 = torch.sum(torch.square(y - torch.matmul(W, lda)))

    # full model: y = Z * lambda + X * beta
    W = torch.cat((torch.ones(n).reshape(-1, 1), Z, X), dim=1)
    # gamma: fitted regression parameters of the full model, where gamma = (lambda, beta)
    gamma = torch.matmul(torch.matmul(torch.pinverse(torch.matmul(W.T, W)), W.T), y)
    SSE1 = torch.sum(torch.square(y - torch.matmul(W, gamma)))

    # F-statistics and its degrees of freedom
    # As F stat is bigger, X is more important to predict y
    F = ((SSE0 - SSE1)/p)/(SSE1/(n-p-k))

    # small pvalue -> favor the full model
    pval = 1 - f.cdf(F, p, n-p-k)

    return F, pval, SSE0, SSE1

def run_GLM(df): # run_GLM(df, race):
    X, Y, Z = get_XYZ(df) # get_XYZ(df, race)

    X = torch.tensor(X.values.astype(np.float64)) # n * p
    Z = torch.tensor(Z.values.astype(np.float64)) # n * k
    glm_fstat, glm_pval, glm_Rsse, glm_Fsse = [], [], [], []

    for y_col in Y.columns:
        y = torch.tensor(Y[y_col].values)/1.0
        F, pval, Rsse, Fsse = GLM(y, Z, X)
        glm_fstat.append(F.item())
        glm_pval.append(pval)
        glm_Rsse.append(Rsse.item())
        glm_Fsse.append(Fsse.item())

    return glm_fstat, glm_pval, glm_Rsse,  glm_Fsse

In [358]:
glm_fstat, glm_pval, glm_Rsse,  glm_Fsse = run_GLM(df_2488)

## Result

In [344]:
# destrieux name list
destrieux = pd.read_csv('/root/yubin/CoCONet/data/fin_UD_destrieux.csv')
destrieux_list = destrieux.columns[1:]

In [347]:
destrieux

,Image-name-unique,Left-G_and_S_frontomargin,Left-G_and_S_occipital_inf,Left-G_and_S_paracentral,Left-G_and_S_subcentral,Left-G_and_S_transv_frontopol,Left-G_and_S_cingul-Ant,Left-G_and_S_cingul-Mid-Ant,Left-G_and_S_cingul-Mid-Post,Left-G_cingul-Post-dorsal,...,Right-S_parieto_occipital,Right-S_pericallosal,Right-S_postcentral,Right-S_precentral-inf-part,Right-S_precentral-sup-part,Right-S_suborbital,Right-S_subparietal,Right-S_temporal_inf,Right-S_temporal_sup,Right-S_temporal_transverse
0,ud_u_base,0.883050,0.776336,0.749082,0.706293,0.828756,0.800664,0.798636,0.809658,0.847723,...,0.725243,1.226765,0.699459,0.756309,0.752592,0.915114,0.756465,0.828877,0.720399,0.736169
1,ud_u_follow,0.810463,0.743534,0.707732,0.677251,0.764819,0.747684,0.746722,0.775781,0.820358,...,0.698113,1.182631,0.672132,0.706779,0.722002,0.885447,0.734464,0.806302,0.695650,0.710132
2,ud_u_delta,2.094692,2.120783,2.115177,2.123249,2.100364,2.107550,2.108249,2.120078,2.124349,...,2.124503,2.113351,2.124373,2.109813,2.122233,2.122839,2.127866,2.127489,2.126064,2.125220
3,ud_d_base,0.886348,0.788919,0.763559,0.715359,0.834643,0.814406,0.809502,0.821661,0.858997,...,0.733117,1.234536,0.710533,0.767014,0.770524,0.922791,0.764330,0.839899,0.727019,0.745702
4,ud_d_follow,0.828747,0.761890,0.721608,0.696048,0.784040,0.768060,0.760059,0.791275,0.835157,...,0.711084,1.194450,0.686853,0.719727,0.737531,0.912685,0.746047,0.821765,0.709197,0.724566
5,ud_d_delta,2.104519,2.124569,2.114783,2.129631,2.109108,2.111900,2.109870,2.122367,2.126660,...,2.127845,2.116006,2.126765,2.111283,2.120658,2.135667,2.130304,2.130402,2.130606,2.128433


In [359]:
result_GLM = pd.DataFrame(
    {'Image-name-unique': destrieux_list, 
    'F-statistic': glm_fstat, 
    'p-value': glm_pval, 
    'R_sse': glm_Rsse,
    'F_sse':  glm_Fsse})
result_GLM

,Image-name-unique,F-statistic,p-value,R_sse,F_sse
0,Left-G_and_S_frontomargin,0.028490,0.865975,32.651696,32.651320
1,Left-G_and_S_occipital_inf,0.213662,0.643953,23.317605,23.315593
2,Left-G_and_S_paracentral,2.454871,0.117288,32.156708,32.124870
3,Left-G_and_S_subcentral,0.024573,0.875449,20.461975,20.461772
4,Left-G_and_S_transv_frontopol,0.002981,0.956460,55.008804,55.008738
...,...,...,...,...,...
143,Right-S_suborbital,0.024269,0.876214,131.939430,131.938137
144,Right-S_subparietal,0.000515,0.981891,18.382601,18.382598
145,Right-S_temporal_inf,1.023099,0.311884,17.313208,17.306059
146,Right-S_temporal_sup,0.476120,0.490250,8.948902,8.947182


In [361]:
pd.set_option('display.float_format', '{:.2e}'.format)

In [362]:
result_GLM[result_GLM['p-value']<0.05]

,Image-name-unique,F-statistic,p-value,R_sse,F_sse
112,Right-Lat_Fis-ant-Horizont,5.86e+00,1.56e-02,4.12e+01,4.11e+01
113,Right-Lat_Fis-ant-Vertical,1.27e+01,3.80e-04,5.63e+01,5.60e+01
122,Right-S_circular_insula_sup,5.99e+00,1.45e-02,1.83e+01,1.83e+01
137,Right-S_orbital-H_Shaped,4.25e+00,3.93e-02,3.52e+01,3.52e+01
